In [3]:
#Import libraries
from datetime import date
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from textblob import  TextBlob
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")

#Scrapping the tweets - Create a list to append tweet data
tweets_list = []
maxTweets = 100

#Using TwitterSearchScraper to scrape data and append tweets to list 
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('ChatGPT').get_items()):
    if i == maxTweets:
        break
    tweets_list.append([tweet.rawContent])

#Creating a dataframe from the tweets list above

tweets_to_df = pd.DataFrame(tweets_list, columns=["Tweets"])

tweets_to_df.head() #list first five tweets

#clean the tweets with a function

def cleanTweets(text):
    text = re.sub('@[A-Za-z0-9_]+', '', text) #remove @mentions
    text = re.sub('#', '', text) #removes hastag '#' symbol
    text = re.sub('RT[\s]+', '',text)
    text = re.sub('https?:\/\/\S+', '', text)
    text = re.sub('\n',' ', text)
    return text
tweets_to_df['cleanedTweets'] = tweets_to_df['Tweets'].apply(cleanTweets) #apply cleanTweet function to the tweet
tweets_to_df.head() #compares original theets with cleaned tweets

,Tweets,cleanedTweets
0,@Flare_NFT @AstralCredits @lolalolabasyang @Tu...,
1,@berkelot1 gn gn :),gn gn :)
2,@gmcafeNFT Cupcatsfan#2053,Cupcatsfan2053
3,We're happy to attend this program. \nI'm so g...,We're happy to attend this program. I'm so gl...
4,🔒 ChatGPTの高度なブロックチェーン技術で投資を安全に守りましょう。 #セキュリティ ...,🔒 ChatGPTの高度なブロックチェーン技術で投資を安全に守りましょう。 セキュリティ 仮...
